In [1]:
import subprocess
import zlib
from xml.dom.minidom import parseString
import json

import pandas as pd
import tqdm

In [2]:
# sas_path = 'ecg_sample/ecg_sample.sas7bdat'
sas_path = 'ecg_sample/ecg_sample_v2.sas7bdat'
csv_path = 'ecg_sample/ECG sample.csv'

# SAS Information

In [3]:
sas_df = pd.read_sas(sas_path)
print(len(sas_df))
sas_df

4051


,patID,ecgId,dateAcquired,dateReceived,dateConfirmed,userField4,severityId,headerInfo,userDefines,orderInfo,documentInfo,reportInfo,acquisitionInfo,interpretationInfo,age,ageUnits,sex,waveformInfo,measurementInfo
0,1.0,b'42DC3D80-F266-11E7-4823-000DB1100029',2018-01-05 15:11:52,2018-01-05 22:39:55.740000010,2018-01-06 22:11:01,b'E',1.0,"b""x\x9c\xad\x91\xcd\n\xc20\x10\x84\xe7Q\x8aw\x...",b'x\x9c\xbdT\xcb\n\xc2@\x0c\x9cOY\xbc\x8b\xb6Z...,b'x\x9c\x85S\xedN\x83@\x10\x9cG!\xfdmS\xb0\xe2...,b'x\x9c\xb5\x93AK\xc3@\x10\x85\xdfO\xd9\x9bzH5...,b'x\x9c\x95U\xd9N\xc3@\x0c\xf4\xa7D\xbcs\xa4\x...,b'x\x9c\xadV]o\xdaP\x0c\xf5O\x89\xf6\xbc\x0e\x...,b'x\x9c\xd5X\xdbn\xda@\x10\xddOA\xbcT\xaaj\x11...,0.0,1.0,2.0,b'x\x9c\xbc\xbc\xd7\x8e\xf3Z\xb6\x9e}^@\xdd\xc...,b'x\x9c\xed\x9d[s\xdb\xba\x11\x80\xf1S<y\xae&&...
1,2.0,b'0BEA3580-F22E-11E7-4823-000EFF2E0029',2018-01-05 08:29:28,2018-01-05 15:30:40.663000107,2018-01-05 23:35:06,b'O',4.0,"b""x\x9c\xad\x91\xcd\n\xc20\x10\x84\xe7Q\x8aw\x...",b'x\x9c\xbdT\xcb\n\xc2@\x0c\x9cOY\xbc\x8b\xd6G...,"b""x\x9c\x85S\xedN\x83@\x10\x9cG!\xfc\xb6) \xf5...",b'x\x9c\xb5\x93AO\xc2@\x10\x85\xdfO\xd9\x9bz\x...,b'x\x9c\x95U\xd9N\xc3@\x0c\xf4\xa7D\xbc\x03Mi\...,b'x\x9c\xadV\xdbn\xda@\x10\x9dO\xb1\xf2\xdc\x0...,b'x\x9c\xadW\xe9n\xda@\x10\xdeGA\xf9Y\t\x01\x8...,0.0,1.0,1.0,b'x\x9c\xac\xbcY\x8f\x83J\x9a&|\x9fR\xfe\x87\x...,b'x\x9c\xed\x9dm\x93\xdb\xb6\x11\x80\xf1Sn\xfc...
2,3.0,b'801BCF00-F183-11E7-4823-001C25190029',2018-01-04 13:08:30,2018-01-04 21:17:58.796999931,2018-01-17 20:24:39,b'O',1.0,"b""x\x9c\xad\x91\xcd\n\xc20\x10\x84\xe7Q\x8aw\x...",b'x\x9c\xbdT]\x0b\x82@\x10\x9c\x9fr\xf4\x1eQj\...,b'x\x9c\x85S\xedN\xc2@\x10\x9cGi\xf8\r\x81*_6G...,b'x\x9c\xb5\x93AK\xc3@\x10\x85\xdfO\xd9\x9bz\x...,b'x\x9c\x95U\xdbN\xe3@\x0c\xf5\xa7D\xbcsI[\x10...,b'x\x9c\xb5V\xd9n\xd3@\x14\xbd\x9fb\xf5\x99\x9...,b'x\x9c\xcdXQs\xdaF\x10\xbe\x9f\xa2\xf1C\x1e:\...,0.0,1.0,1.0,b'x\x9c\xac\xbcW\x8f\xf3\xc8\x9a\xa5{\x9f@\xfe...,b'x\x9c\xed\x9dms\xe3\xb6\x11\x80\xf1S<\xf9\\M...
3,4.0,b'1EC6E680-EF5E-11E7-4823-00038A9E0029',2018-01-01 18:35:58,2018-01-02 02:56:14.306999922,2018-01-11 07:09:52,b'E',4.0,b'x\x9c\xad\x91\xcd\n\xc20\x10\x84\xe7Q\x8aw\x...,b'x\x9c\xbdS\xcb\n\xc20\x10\x9cO\t\xfe\x80\xb6...,b'x\x9c\x85S]O\x83P\x0c=?\x85\xecY3`\x1fN\x82\...,b'x\x9c\xb5\x93AO\xc2@\x10\x85\xdfO\xd9\x9bz\x...,b'x\x9c\x95U]O\xdbP\x0c\xf5O\x89\xfa\xbeAZ\x8a...,b'x\x9c\xadV\xcbn\xdaP\x10\x9dO\xb1X7\xe1\x11A...,b'x\x9c\xedYmo\xe2F\x10\xde\x9fb\xf1\xa1\x1f\x...,0.0,1.0,2.0,b'x\x9c\xac\xbcW\x8f\xeb<\x97&z\xbf\x81\xfa\x0...,b'x\x9c\xed\x9dmo\xe4\xb6\x11\x80\xf9S\x8c\xfb...
4,5.0,b'58643280-F0C1-11E7-4823-0007E1690029',2018-01-03 12:58:48,2018-01-03 19:59:41.806999922,2018-01-18 17:04:55,b'I',3.0,"b""x\x9c\xad\x91\xcd\n\xc20\x10\x84\xe7Q\x8aw\x...",b'x\x9c\xbdT\xd9\n\xc2@\x0c\x9cOY|\x17\x8f\xb6...,"b""x\x9c\x85S\xd1N\x83@\x10\x9cO!}\xb6)\xd46\x1...",b'x\x9c\xb5\x93OK\xc3@\x10\xc5\xdfG\xd9\x9bzhm...,b'x\x9c\x95U\xdbN\xc2@\x10\x9dOi|\xf7\x02\x821...,b'x\x9c\x95V\xdbn\xda@\x10\x9dO\xb1\xfa\xdc\x9...,b'x\x9c\xedX\xdbr\xd3H\x10\x9dOQ\xe5a\x1f(\x8c...,0.0,1.0,1.0,b'x\x9c\xd4\xbcY\x8f\xb3\\\x97\xa6y\xfeH\xf1\x...,b'x\x9c\xed\x9d\xebs\xdb\xb8\x11\xc0\xf1\xa7x\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4046,2947.0,b'E938F200-EF8B-11E7-4823-002D115F0029',2018-01-02 00:03:49,2018-01-02 07:04:33.322999954,2018-01-04 15:53:57,b'E',2.0,"b""x\x9c\xad\x91\xcd\n\xc20\x10\x84\xe7Q\x8aw\x...",b'x\x9c\xbdT\xd9\n\xc2@\x0c\x9cOY|\x17i\xabV\x...,"b""x\x9c\x85S\xd1N\x83@\x10\x9cO!}\xb6) ZB\xae$...","b""x\x9c\xb5\x93OK\xc3@\x10\xc5\xdfG\xd9\x9bzh\...",b'x\x9c\x95U\xdbN\xc30\x0c\xf5\xa7T\xbcs\xe9.\...,b'x\x9c\x95V\xdbn\xd3@\x10\x9dO\xb1x\xa6\xe4\x...,b'x\x9c\xedX[o\xdaH\x14>?\xc5\xcaC\x1fVB\x04\x...,0.0,1.0,2.0,b'x\x9c\xd4\xbc\xd7\x8e\xeb\xca\xb6\xa6y\x9f@\...,b'x\x9c\xed\x9dmo\xe4\xb6\x11\x80\xf9S\x8c\xfb...
4047,2948.0,b'BA932B00-EF3B-11E7-4823-0007B51D0029',2018-01-01 14:29:48,2018-01-01 21:33:01.667000055,2018-01-03 16:50:32,b'I',2.0,"b""x\x9c\xad\x91\xcd

In [4]:
sas_df.iloc[0]

patID                                                                 1
ecgId                           b'42DC3D80-F266-11E7-4823-000DB1100029'
dateAcquired                                        2018-01-05 15:11:52
dateReceived                              2018-01-05 22:39:55.740000010
dateConfirmed                                       2018-01-06 22:11:01
userField4                                                         b'E'
severityId                                                            1
headerInfo            b"x\x9c\xad\x91\xcd\n\xc20\x10\x84\xe7Q\x8aw\x...
userDefines           b'x\x9c\xbdT\xcb\n\xc2@\x0c\x9cOY\xbc\x8b\xb6Z...
orderInfo             b'x\x9c\x85S\xedN\x83@\x10\x9cG!\xfdmS\xb0\xe2...
documentInfo          b'x\x9c\xb5\x93AK\xc3@\x10\x85\xdfO\xd9\x9bzH5...
reportInfo            b'x\x9c\x95U\xd9N\xc3@\x0c\xf4\xa7D\xbcs\xa4\x...
acquisitionInfo       b'x\x9c\xadV]o\xdaP\x0c\xf5O\x89\xf6\xbc\x0e\x...
interpretationInfo    b'x\x9c\xd5X\xdbn\xda@\x10\xddOA\xbcT\xaaj

In [5]:
def xxd_to_zlib_flate(raw_data, skip_fallback=True):
    """
    convert the sql hexadecimal values into raw xml representation
    """
    # assert raw_data_str.startswith("0x")
    try:
        zlib_out = zlib.decompress(raw_data)
    except Exception as e:
        zlib_out = b''

    if zlib_out or skip_fallback:
        return zlib_out
    
#     print(raw_data)
    # fallback to subprocess and zlib-flate, truncate errors
    xxd = subprocess.Popen(("xxd", "-r", "-p"), stdin=subprocess.PIPE,
                           stderr=subprocess.STDOUT, stdout=subprocess.PIPE)
    xxd_out = xxd.communicate(input=raw_data)[0]
    zlib_flate = subprocess.Popen(
        ("zlib-flate", "-uncompress"),
        stdin=subprocess.PIPE, stderr=subprocess.STDOUT, stdout=subprocess.PIPE)
    zlib_out = zlib_flate.communicate(input=xxd_out)[0]
    
    return zlib_out


In [6]:
info_keys = [k for k in sas_df.keys() if k.endswith("Info")]
counter = 0
error_counter = {}

for i in tqdm.trange(len(sas_df)):
    errors = []
    raw_outputs = []
    for info_key in info_keys:
        raw_data = sas_df.iloc[i][info_key]
        # print(info_key, f"{len(raw_data)} bytes")
        try:
            output_data = xxd_to_zlib_flate(raw_data).decode("utf-16")
            if not output_data:
                raise ValueError("output_data should be truthy")
        except Exception as e:
            errors.append(info_key)
            output_data = None
        raw_outputs.append((info_key, output_data))

    for error in errors:
        error_counter[error] = error_counter.get(error, []) + [sas_df.iloc[i].ecgId.decode("ASCII")]

print("Errors:")
for k, v in error_counter.items():
    print(k, len(v))


100%|██████████| 4051/4051 [00:08<00:00, 451.55it/s]

Errors:
waveformInfo 4051
acquisitionInfo 34
orderInfo 19
interpretationInfo 27
measurementInfo 26
documentInfo 27
reportInfo 1


In [7]:
sorted_error_counter = dict(sorted([(k, v) for (k, v) in error_counter.items()], key=lambda tup: len(tup[1])))

# print(sorted_error_counter)

with open("ecg_sample_decode_errors.json", "w") as f:
    json.dump(sorted_error_counter, f, indent=4)

In [13]:
sas_row = sas_df.iloc[3]
errors = []
raw_outputs = []

for info_key in info_keys:
    raw_data = sas_row[info_key]
    print(info_key, f"{len(raw_data)} bytes")
    try:
        output_data = xxd_to_zlib_flate(raw_data, skip_fallback=True).decode("utf-16")
    except Exception as e:
        print("\t", info_key, e)
        errors.append(info_key)
        output_data = None
    raw_outputs.append((info_key, output_data))

print("=" * 40)
for key, value in raw_outputs:
    try:
        print(key)
        print(parseString(value).toprettyxml())
    except Exception:
        print("ERROR:", value)
    print()


headerInfo 203 bytes
orderInfo 337 bytes
documentInfo 320 bytes
reportInfo 573 bytes
acquisitionInfo 701 bytes
interpretationInfo 1348 bytes
waveformInfo 8000 bytes
measurementInfo 4707 bytes
headerInfo
ERROR: <restingecgdata xsi:schemaLocation="http://www3.medical.philips.com PhilipsECG.xsd" status="Confirmed" lang="en" locale="US" xmlns="http://www3.medical.philips.com" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">

orderInfo
<?xml version="1.0" ?>
<orderinfo priority="" orderrequestdate="2018-01-01" orderrequesttime="19:56:14" dateprocessed="2018-01-01" timeprocessed="20:23:16">
	<ordernumber>20180101195614052</ordernumber>
	<uniqueorderid>1514861792739</uniqueorderid>
	<orderbillingcode/>
	<orderremarks>StationName:PWTC70_FSH750,UTC:-420</orderremarks>
	<reasonfororder/>
	<drgcategories>
		<drgcategory id=""/>
	</drgcategories>
	<orderstatus>RECONCILED</orderstatus>
	<inbox>FSH</inbox>
</orderinfo>


documentInfo
<?xml version="1.0" ?>
<documentinfo>
	<documentname>1ec6e68

# CSV information

In [ ]:
csv_df = pd.read_csv(csv_path)
print(len(csv_df))
# csv_df

In [ ]:
csv_df.iloc[0]

In [ ]:
csv_df.iloc[0].waveformInfo  # should be defined